# Cell Shader
### By Snowy
*IMPORTANT: all inputs must be in ".jpg" format

---


1. First, we import the necessary libraries: `cv2` for image processing, `numpy` for numerical operations, `PIL` for image manipulation, `ipywidgets` for creating interactive widgets, and `IPython.display` for displaying the output.

2. We define the path to the input image file.

3. The `image` variable is used to load the input image using `Image.open()` from the PIL library.

4. We create a `FloatSlider` widget named `threshold_slider` to adjust the threshold value. The slider ranges from 0.0 to 1.0 with a step size of 0.01. It is initially set to a value of 1.0. This slider will be used to control the threshold for the edge detection process.

5. An output widget named `output_image` is created. This widget will be used to display the cell-shaded image.

6. The `update_cell_shading()` function is defined. This function takes the threshold value as input and updates the cell-shaded image based on that value. Here's what it does:
   - Convert the input image to grayscale using the `convert()` method from PIL.
   - Apply color quantization to reduce the number of colors in the image using the `posterize()` method from PIL. The number of colors is set to 8.
   - Perform edge detection using the Canny algorithm from the `cv2` library. The threshold value is scaled to the range 0-255 and applied to the `Canny()` function.
   - Clear the output widget and display the updated cell-shaded image using the `Image.fromarray()` method from PIL.

7. The `interactive()` function from `ipywidgets` is used to link the `update_cell_shading()` function with the `threshold` parameter of the `threshold_slider` widget. This ensures that the `update_cell_shading()` function is called whenever the slider value changes.

8. We display the `threshold_slider` and `output_image` widgets using the `display()` function from `IPython.display`.

9. Finally, we initialize the cell-shaded image by calling the `update_cell_shading()` function with the initial value of the `threshold_slider`.

With this code, you can interactively adjust the threshold value using the slider, and the cell-shaded image will update in real-time based on the selected threshold. The number of colors in the cell-shaded image is fixed at 8 due to the threshold slider becoming unable to function properly

In [ ]:
#@title Install
!pip install opencv-python

In [ ]:
#@title Import
import cv2
import numpy as np
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import files
from PIL import Image
import io
import os

# Create the "input" directory if it doesn't exist
input_dir = "/content/input"
os.makedirs(input_dir, exist_ok=True)

# Upload the image file
uploaded = files.upload()

# Get the file name of the uploaded image
file_name = next(iter(uploaded))

# Set the desired file name for the image
new_file_name = "image.jpg"

# Load the image
image = Image.open(io.BytesIO(uploaded[file_name]))

# Check if the image was loaded successfully
if image is None:
    print("Failed to load the image.")
else:
    # Save the original image in the "content" folder
    image.save(os.path.join("/content", file_name))
    
    # Move the image to the "input" folder with the desired file name
    os.rename(os.path.join("/content", file_name), os.path.join(input_dir, new_file_name))
    
    print("Image loaded successfully.")


In [ ]:
#@title Load Image
# Load the input image from the "input" folder
image_path = 'input/image.jpg'
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    print("Failed to load the image.")
else:
    # Display the input image
    cv2_imshow(image)


In [ ]:
import cv2
import numpy as np
from PIL import Image, ImageOps
import ipywidgets as widgets
from IPython.display import display
from google.colab import files
import os

# Define the global variables for image and edges
image = None
edges = None

# Load the input image
image_path = 'input/image.jpg'
image = Image.open(image_path)

# Create a slider widget for adjusting the threshold value
threshold_slider = widgets.FloatSlider(value=1.0, min=0.0, max=1.0, step=0.01, description='Threshold:')

# Create an output widget for displaying the cell-shaded image
output_image = widgets.Output()

# Define a function to update the cell-shaded image based on the threshold value
def update_cell_shading(threshold):
    global edges, image  # Make edges and image variables global

    # Convert the image to grayscale
    grayscale = image.convert('L')

    # Apply color quantization to reduce the number of colors
    posterized = ImageOps.posterize(grayscale, 8)

    # Apply edge detection using the Canny algorithm
    edges = cv2.Canny(np.array(posterized), threshold * 255, threshold * 255 * 2)

    # Clear the output widget and display the updated cell-shaded image
    with output_image:
        output_image.clear_output()
        display(Image.fromarray(edges, 'L'))

# Register the update function to be called when the slider value changes
widgets.interactive(update_cell_shading, threshold=threshold_slider)

# Display the slider and the initial cell-shaded image
display(threshold_slider)
display(output_image)

# Initialize the cell-shaded image with the default slider value
update_cell_shading(threshold_slider.value)

# Create the "output" directory if it doesn't exist
output_dir = "/content/output"
os.makedirs(output_dir, exist_ok=True)

# Define a function to handle the download of the cell-shaded image
def download_image(_):
    global edges  # Make edges variable global

    if edges is not None:
        # Save the cell-shaded image as "out.jpg"
        out_image_path = os.path.join(output_dir, "out.jpg")
        Image.fromarray(edges, 'L').save(out_image_path)
        print("Cell-shaded image saved as 'out.jpg'")
    else:
        print("Cell-shaded image not available. Please adjust the threshold and try again.")

# Create the download button
download_image_button = widgets.Button(description="Download as Image")

# Register the button click event
download_image_button.on_click(download_image)

# Display the download button
display(download_image_button)
